In [1]:
import sys
sys.path.append('/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/')
import importlib
# import try6
# importlib.reload(try6)
# import try6
# from try6 import *

# import try6_wordLevel
# importlib.reload(try6_wordLevel)
# import try6_wordLevel
# from try6_wordLevel import *

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log

# Given floors from CHI'18
mu, sigma = 0.0117, 0.0143  # mean and std (as proportions)

# Quantile anchors
n1, n2 = 19, 40

# Desktop (physical keyboard) bump targets for i=1 (proportions)
scenarios = {
    "Conservative": {"Delta1": 0.050, "Delta2": 0.010},
    "Balanced":     {"Delta1": 0.010, "Delta2": 0.012},
    "Aggressive":   {"Delta1": 0.015, "Delta2": 0.015},
}

def solve_a_b(n1, n2, d1, d2):
    # Solve for a, b from two anchors for Δ(n) = (a + b ln n)/n
    b = (d1*n1 - d2*n2) / (np.log(n1) - np.log(n2))
    a = d1*n1 - b*np.log(n1)
    return a, b

def cer_option_B(n, i, a, b):
    # Option B: CER(n|i) = mu + i*sigma + i * (a + b ln n) / n
    return 100 *(mu + i*sigma + i * (a + b*np.log(n+5)) / n), [n * (mu + i*sigma + i * (a + b*np.log(n+20)) / n) for n in [5, 10, 20, 50, 100]]

# Solve for (a,b) per scenario and store
rows = []
for name, vals in scenarios.items():
    a, b = solve_a_b(n1, n2, vals["Delta1"], vals["Delta2"])
    rows.append({"Scenario": name, "a": a, "b": b, "Δ(50)": vals["Delta1"], "Δ(80)": vals["Delta2"]})
params_df = pd.DataFrame(rows)

# Prepare n range
n = np.logspace(0, 4, 300)  # 10 to 10,000 (log scale)

# Plot for each scenario: Low(i=1), Medium(i=2), High(i=3)
for name, vals in scenarios.items():
    a, b = solve_a_b(n1, n2, vals["Delta1"], vals["Delta2"])


    plt.figure(figsize=(9,6))
    for i in [1, 2, 3]:
        cer_vals, examplars = cer_option_B(n, i, a, b)
        print(examplars)

    #     label = f"Tier i={i}"
    #     plt.plot(n, cer_vals, label=label)
    # plt.xscale("log")
    # plt.xlabel("Text length n (log scale)")
    # plt.ylabel("Expected CER (%)")
    # plt.title(f"Hybrid CER — {name} (Desktop); anchors: Δ(50)={vals['Delta1']*100:.1f}pp, Δ(80)={vals['Delta2']*100:.1f}pp")
    # plt.grid(True, which="both", linestyle=":")
    # plt.legend(title="Intensity")
    # plt.show()


[0.8772433388356351, 0.8725423659422049, 0.9199999999999999, 1.2865503103036, 2.188334354321511]
[1.6959866776712704, 1.6280847318844098, 1.6059999999999999, 1.9881006206072, 3.206668708643022]
[2.5147300165069058, 2.3836270978266145, 2.292, 2.6896509309108, 4.225003062964533]
[0.42690805770484663, 0.6279322070486555, 0.9999999999999998, 1.9980007454762831, 3.507969158630476]
[0.7953161154096933, 1.138864414097311, 1.7659999999999996, 3.411001490952567, 5.845938317260951]
[1.16372417311454, 1.6497966211459665, 2.531999999999999, 4.82400223642885, 8.183907475891427]
[0.5311242695759543, 0.7382711904149192, 1.12, 2.1367939131897566, 3.664863051615862]
[1.0037485391519085, 1.3595423808298381, 2.0060000000000002, 3.688587826379512, 6.159726103231724]
[1.4763728087278627, 1.9808135712447572, 2.8920000000000003, 5.240381739569269, 8.654589154847587]


<Figure size 900x600 with 0 Axes>

<Figure size 900x600 with 0 Axes>

<Figure size 900x600 with 0 Axes>

In [5]:
import sys
sys.path.append('/Users/keqiaoli/Desktop/RankingSHAP/RankingShap/')
import importlib
# import importlib
# importlib.reload(try6)
# import try6
# from try6 import *
import perturbation_char_level
importlib.reload(perturbation_char_level)
import perturbation_char_level
from perturbation_char_level import *


In [6]:
# for i in range(1, 2000):
#     n_edits = perturbation_char_level._expected_edits(i, intensity="low")
# # perturbation_char_level._expected_edits(1000, intensity="low")  # 2.86

In [7]:

# Example usage
def example_usage():
    """Example of how to use the testing framework"""
    
    # Mock model interface (replace with your actual model)
    def mock_model(query):
        # Simulate retrieval results
        # In real usage, this would call your BM25 + dense search + cross-encoder pipeline
        results = [
            (f"doc_{i}", 1.0 - i * 0.1) 
            for i in range(10)
        ]
        
        # Simulate sensitivity to perturbations
        if query != query.lower():  # Case sensitive
            results = results[1:] + [("doc_extra", 0.1)]
        
        return results
    
    # Initialize tester
    tester = HybridIRModelTester(mock_model)
    
    # Test queries
    test_queries = [
        "artificial intelligence",
        "card",
        "IRA",
        "what's my credit card number?",
        "machine learning algorithms",
        "natural language processing",
        "information retrieval systems",
        "deep learning neural networks"
        "card number: 1234-5678-9012-3456",
    ]
    
    # Generate test cases
    print("Generating test cases...")
    test_cases = tester.generate_test_cases(
        queries=test_queries,
        perturbation_types={
            PerturbationType.TYPO: ["low", "moderate", "high"],
            PerturbationType.DELETION: ["low", "moderate", "high"],
            # PerturbationType.CASE_CHANGE: [0.05, 0.1, 0.2, 0.3, 0.5],
            # # PerturbationType.UNICODE_SUBSTITUTION: [0.05, 0.1, 0.2, 0.3, 0.5],
            PerturbationType.INSERTION: ["low", "moderate", "high"],
            # # PerturbationType.SUBSTITUTION: [0.05, 0.1, 0.2, 0.3, 0.5],
            PerturbationType.TRANSPOSITION: ["low", "moderate", "high"],
            # PerturbationType.DUPLICATION: [0.05, 0.1, 0.2, 0.3, 0.5],
            # PerturbationType.CASE_CHANGE: [0.05, 0.1, 0.2, 0.3, 0.5],
            # PerturbationType.WHITESPACE_NOISE: [0.05, 0.1, 0.2, 0.3, 0.5],
            # PerturbationType.PUNCTUATION_NOISE: [0.05, 0.1, 0.2, 0.3, 0.5]
        },
        # intensity_levels=[0.05, 0.1, 0.2, 0.3, 0.5]
    )
    # print(test_cases)

    for test_case in test_cases:
        print(f"Test Case: {test_case.original_query} -> {test_case.perturbed_query} ({test_case.perturbation_type.value}, {test_case.intensity})")



In [8]:
example_usage()

Generating test cases...
Test Case: artificial intelligence -> artificisl intelligence (typo, low)
Test Case: artificial intelligence -> artificizl intelligence (typo, moderate)
Test Case: artificial intelligence -> artificial intell8gence (typo, high)
Test Case: artificial intelligence -> artificial intlligence (deletion, low)
Test Case: artificial intelligence -> artificil intelligence (deletion, moderate)
Test Case: artificial intelligence -> artificial intellignce (deletion, high)
Test Case: artificial intelligence -> artificial int5elligence (insertion, low)
Test Case: artificial intelligence -> arrtificial intelligence (insertion, moderate)
Test Case: artificial intelligence -> artificcial intelligence (insertion, high)
Test Case: artificial intelligence -> artificial intleligence (transposition, low)
Test Case: artificial intelligence -> aritficial intelligence (transposition, moderate)
Test Case: artificial intelligence -> artificial intelilgence (transposition, high)
Test Case

In [5]:
text = ("card number: 1234-5678-9012-3456")
spans = []  # (start, end, tok)
for m in TOKEN_RX.finditer(text):
    start, end = m.span()
    tok = text[start:end]
    spans.append((start, end, tok))
for start,end, token in spans:
    print(start,end, token)

0 4 card
5 11 number
11 12 :
13 17 1234
17 18 -
18 22 5678
22 23 -
23 27 9012
27 28 -
28 32 3456


In [132]:
# ---- tunables (adjust to taste) ----
beta0 = 0.6          # baseline edits even for short strings
beta1 = 0.8          # sublinear growth vs. length (log-scale)
max_cer = 0.08       # hard cap on edits as a fraction of length
min_len_for_forced_edit = 4  # don't force edits for very short strings
boundary_skip_p = 0.7 # avoid first/last char of a word ~70% of the time
# ------------------------------------


# Expected edits grow ~ beta0 + beta1*log(len); scaled by intensity
# def _expected_edits(n: int, intensity: float) -> float:
#     if n <= 1:
#         return 0.0
#     return (beta0 + beta1 * math.log(n)) * max(0.05, float(intensity))


# def _expected_edits(n: int, intensity: float) -> float:
#     if n <= 1:
#         return 0.0
#     # log + light linear blend
#     # a, b, k_lin = 0.6, 0.8, 0.02
#     # lam = (a + b * math.log(n)) * intensity + k_lin * n * intensity
#     # lam = intensity * (-1.80 + 1.00 * math.log(n) + 0.10 * math.sqrt(n))
#     lam = intensity * (-1.50 + 1.09 * math.log(n))

#     return lam
# # # Expected edits grow ~ beta0 + beta1*log(len); scaled by intensity
def _expected_edits(n: int, intensity) -> float:
    if n <= 1:
        return 0.0
    # Ensure expected edits never decrease as n increases

    mult = {"low": 0.6, "medium": 1.0, "high": 1.6}.get(intensity, 1.0)
    B: float = 1.5
    n0: int = 30
    max_cer: float = 0.05
    # Core curve: sublinear growth; damped for ultra-short by n0
    lam = mult * B * math.log(1.0 + n / float(n0))
    return lam

    # val = (beta0 + beta1 * math.log(max(n, 2))) * max(0.05, float(intensity))
    # return max(val, n * 0.05 * intensity)  # always grows with n

In [ ]:

def _choose_positions(s: str, 
                      n_edits: int, 
                      max_per_word: int = 2,
                      boundary_skip_p: float = 0.8,
                      perturbation_type: str = "deletion") -> List[int]:
    """
    Select up to n_edits character positions in s to perturb.
    No word contributes more than max_per_word positions.

    boundary_skip_p: probability to skip a boundary character within a word (start/end index).
    """
    if n_edits <= 0 or max_per_word <= 0:
        return []

    # 1) Find non-space "words" as spans (handles multiple spaces/tabs cleanly)

    spans = []  # (start, end, tok)
    for m in TOKEN_RX.finditer(s):
        start, end = m.span()
        tok = s[start:end]
        spans.append((start, end, tok))

    # 2) Collect candidate positions per word
    per_word_positions = []  # list[list[int]]
    for start, end, tok in spans:
        if end <= start:
            per_word_positions.append([])
            continue

        if  _is_mostly_numeric(tok, level = 0.6):
            per_word_positions.append([])
            continue
        if is_punctuation(tok):
            per_word_positions.append([])
            continue

        # Helper: is deleting idx going to zero out its token?
        def _would_zero_token(idx: int, start_idx: int, end_idx: int) -> bool:
            if start_idx <= idx < end_idx:
                # token length after deletion
                new_len = (end_idx - start_idx) - 1
                return new_len <= 0
            return False  # idx not in a token (e.g., whitespace between tokens)


        positions = []
        for i in range(start, end):
            ch = s[i]
            if ch.isspace():
                continue
            if _would_zero_token(i, start, end) and perturbation_type == "deletion":
                continue

            at_boundary = (i == start) or (i == end - 1)

            # Optionally downweight very short tokens
            if len(tok) <= 2 and random.random() < 0.7:
                continue

            # Optionally skip boundaries
            if at_boundary and random.random() < boundary_skip_p:
                continue

            positions.append(i)

        # 3) Shuffle and hard-cap per word to avoid oversampling later
        if positions:
            random.shuffle(positions)
            per_word_positions.append(positions[:max_per_word])
        else:
            per_word_positions.append([])
            
    random.shuffle(per_word_positions)
    # 4) Round-robin selection for fairness, respecting global n_edits
    total_available = sum(len(lst) for lst in per_word_positions)
    target = min(n_edits, total_available)

    selected = []
    round_idx = 0
    while len(selected) < target:
        progressed = False
        for lst in per_word_positions:
            if round_idx < len(lst):
                selected.append(lst[round_idx])
                if len(selected) >= target:
                    break
                progressed = True
        if not progressed:
            break
        round_idx += 1

    # Positions are unique by construction; return as-is
    return selected